# 1. Define your Diff function in python.

In [12]:
import math
import random
import matplotlib.pyplot as plt
import numpy as np
random.seed(42)


# pick three vectors corresponding to letter of each group members name
u = (59, 94)
v = (62, 60)
w = (10, 40)


# define distance function 
def distance(vector_1, vector_2):
    ''' this function takes in two vectors and calculates and returns the distance between them'''
    
    return math.log(3 + 3 * (vector_1[0] - vector_2[0]) ** 2 + 1.5 * (vector_1[1] - vector_2[1]) ** 2)


# define the overall difference function
def difference(input_vector):
    ''' this function takes in a vector, compares it to the three fixed vectors for our group
    and returns the overall difference '''
    
    return distance(input_vector, u) + distance(input_vector, v) + distance(input_vector, w)


# 2. Compute Diff(x) for 20 random points. 
**What is the average value of Diff?**<br>
The average value was 24.91

**What is the lowest and highest value you found?**<br>
Lowest value was 21.63 and the highest was 27.21

**Is random search a good way to minimize this function?**<br>
No. Random search literally just gives you random points that you can then test the value of. There is no learning from the results of previously tried points and the time to get to a relatively good solution could take a very long time. There is no order and you are not "approaching" the minimal value, you are hoping that you happen to get a value that is close to the minimum by pure chance.

In [17]:
# randomly generate 20 points within the range
random_points = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(20)]

# obtain the difference values for each of the 20 randomly generated points
diff_values = [difference(point) for point in random_points]

# obtain and print the average, highest and lowest value
average_diff = np.mean(diff_values)
max_diff = np.max(diff_values)
min_diff = np.min(diff_values)
print(f'Average difference: {average_diff}\nMaximum value: {max_diff}\nMinimum value: {min_diff}')


Average difference: 24.913858638146543
Maximum value: 27.214040207685674
Minimum value: 21.63492481784673


# 3. Set xzero=(50,50). Compute Diff(x)=Diff(50,50).

In [18]:
# create xzero and test with the difference function
xzero = (50, 50)
xzero_diff = difference(xzero)
print(xzero_diff)


22.93451831163977


# 4. Create a function to compute and print the gradient of Diff by using a small delta=0.001.

In [29]:
# create a function that calculates the gradient of the difference
def gradient_diff(input_vector, delta = 0.001):
    ''' this function takes in an input_vector and calculates its gradient in the 
    difference function '''
    
    # alter x_1 by delta then calculate the first partial derivative (gradient of x1)
    altered_x1_vector = (input_vector[0] + delta, input_vector[1])
    x1_gradient = (difference(altered_x1_vector) - difference(input_vector)) / delta
    
    # alter x_2 by delta then calculate the second partial derivative (gradient of x2)
    altered_x2_vector = (input_vector[0], input_vector[1]  + delta)
    x2_gradient = (difference(altered_x2_vector) - difference(input_vector)) / delta

    return (x1_gradient, x2_gradient)
    


# 5. Use this function to print the gradient at (0,0), (100,0), (0,100) and (100,100). 
**Based on these values, is it likely that the minimum of the function is inside this area? And the maximum?**

# 6. Define xzero=(50,50) and stepsize=1.0. 
**Make a function to compute xnext by taking e a small step into the opposite direction of the gradient, so that diff decreases. E.g. if the gradient is (0.4,0.6), do xnext=(oldx1-0.4*stepsize,oldx2-0.6*stepsize**

# 7. Repeat this multiple times (make a function to do this automatically) and print x and Diff(x) at each step. 
**You may need to adjust stepsize if you overshoot or are not moving at all. Try to reach a local minimum.**

# 8. Repeat step 7 with starting points (0,0) and (100,100). 
**Do you always end up at the same point?**

# 9. Make the nicest possible chart using mathplotlib that shows what the Diff function looks like.

# 10. Show the gradient field as well in a chart. 
**Use a new plot or add the gradient to the original plot.**